<h1 align="center">Space-Lunch</h1>
A journey through coding with the Engineering Process

![EDP](https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fwww.sciencebuddies.org%2Fblog%2Fgraphics%2F2013-blog-scientific-method-engineering-design-charts-750px.png&f=1&nofb=1)

# Defining our Problem

## Abstract
Find places to eat near SpaceX launch sites.

## Objective
Make a Jupyter Notebook application that will find places to eat near user desired SpaceX launch site and display them on a map.

## Requirements
1. Allow user to determine which SpaceX launch they want to visit
2. Display lunch options within reasonable driving distance on the map.

## Optional:
1. Aloow user to specify cuisine types for results.

## Project Steps:
1. [x] Get a list of the upcoming launches and include the date and location. (Data source: Probably SpaceX API)
    1. [x] Get the date and launchpad id of upcoming launches from the upcoming launches route.
    2. [x] Get the location for each launch pad.
2. [x] Display the list of launches to the user.
3. [x] Get the user input on which launch to attend. (Consider if we can ask for cuisine preference here.)
4. Take the location of that launch and find restaurants near it. (Data Source: probably Googple maps API)
    1. Determine how to convert SpaceX launch location to google required format
    2. Investigate requird radius size for search.
5. Clean Google API data.
6. display cleaned data to user on a map.

## Research Data:
Most of our research will be on the data sources we intend to use, so we can link to the documentation for that here:

1. [SpaceX upcoming launch docs](https://github.com/r-spacex/SpaceX-API/blob/master/docs/launches/v5/upcoming.md)
2. [SpaceX launch pad docs](https://github.com/r-spacex/SpaceX-API/blob/master/docs/launchpads/v4/one.md)

# Prototyping
This section of the notebook is designated just for prototyping and testing out pieces of code. Much of our code is "test as you go", testing out each each step in the project as we program it, but this section set aside for all of our messy experimentation while we try to figure out what kind of approach we want to take to building each step.

## Get a list of the upcoming launches and include the date and location. (Data source: Probably SpaceX API)

In [16]:
# import dependencies
import requests
import json
import pandas as pd
import pprint

In [5]:
url = "https://api.spacexdata.com/v5/launches/upcoming"
launch_data = requests.get(url).json()
pprint.pprint(launch_data)

[{'auto_update': True,
  'capsules': [],
  'cores': [{'core': None,
             'flight': None,
             'gridfins': None,
             'landing_attempt': None,
             'landing_success': None,
             'landing_type': None,
             'landpad': None,
             'legs': None,
             'reused': None}],
  'crew': [],
  'date_local': '2022-10-20T10:50:00-04:00',
  'date_precision': 'hour',
  'date_unix': 1666277400,
  'date_utc': '2022-10-20T14:50:00.000Z',
  'details': None,
  'failures': [],
  'fairings': {'recovered': None,
               'recovery_attempt': None,
               'reused': None,
               'ships': []},
  'flight_number': 188,
  'id': '63161345ffc78f3b8567070d',
  'launch_library_id': 'adcb0090-6f97-40c2-821f-104b5ed3e9c7',
  'launchpad': '5e9e4501f509094ba4566f84',
  'links': {'article': None,
            'flickr': {'original': [], 'small': []},
            'patch': {'large': 'https://images2.imgbox.com/e3/cc/hN96PmST_o.png',
               

In [14]:
dates = []
IDs = []
lats = []
longs = []
locations = []

# For each launch in in our list:
for launch in launch_data:
    # Get the launch date
    date = launch['date_local']
    print(date)
    # Get the launchpad ID
    ID = launch['launchpad']
    print(ID)
    url = f"https://api.spacexdata.com/v4/launchpads/{ID}"
    launchpad_data = requests.get(url).json()

    # Get location data for each launchpad ID
    lat = launchpad_data["latitude"]
    long = launchpad_data["longitude"]
    location = launchpad_data["full_name"]

    dates.append(date)
    IDs.append(ID)
    lats.append(lat)
    longs.append(long)
    locations.append(location)
    print(f"{lat}, {long}")
    print(location)
    print()

2022-10-20T10:50:00-04:00
5e9e4501f509094ba4566f84
28.5618571, -80.577366
Cape Canaveral Space Force Station Space Launch Complex 40

2022-10-08T19:05:00-04:00
5e9e4501f509094ba4566f84
28.5618571, -80.577366
Cape Canaveral Space Force Station Space Launch Complex 40

2022-10-15T01:22:00-04:00
5e9e4501f509094ba4566f84
28.5618571, -80.577366
Cape Canaveral Space Force Station Space Launch Complex 40

2022-10-31T06:44:00-04:00
5e9e4502f509094188566f88
28.6080585, -80.6039558
Kennedy Space Center Historic Launch Complex 39A

2022-11-04T20:00:00-04:00
5e9e4501f509094ba4566f84
28.5618571, -80.577366
Cape Canaveral Space Force Station Space Launch Complex 40

2022-11-08T19:00:00-05:00
5e9e4501f509094ba4566f84
28.5618571, -80.577366
Cape Canaveral Space Force Station Space Launch Complex 40

2022-11-14T19:00:00-05:00
5e9e4501f509094ba4566f84
28.5618571, -80.577366
Cape Canaveral Space Force Station Space Launch Complex 40

2022-11-18T17:00:00-05:00
5e9e4502f509094188566f88
28.6080585, -80.6039

## Display the list of launches to the user.

In [23]:
# Create a dataframe for the launches
df = pd.DataFrame(
    {
        "date": dates,
        "ID": IDs,
        "lat": lats,
        "long": longs,
        "location": locations
    }
)
df['time'] = df['date'].apply(lambda x: x.split("T")[1])
df['date'] = df['date'].apply(lambda x: x.split("T")[0])
df

,date,ID,lat,long,location,time
0,2022-10-20,5e9e4501f509094ba4566f84,28.561857,-80.577366,Cape Canaveral Space Force Station Space Launc...,10:50:00-04:00
1,2022-10-08,5e9e4501f509094ba4566f84,28.561857,-80.577366,Cape Canaveral Space Force Station Space Launc...,19:05:00-04:00
2,2022-10-15,5e9e4501f509094ba4566f84,28.561857,-80.577366,Cape Canaveral Space Force Station Space Launc...,01:22:00-04:00
3,2022-10-31,5e9e4502f509094188566f88,28.608058,-80.603956,Kennedy Space Center Historic Launch Complex 39A,06:44:00-04:00
4,2022-11-04,5e9e4501f509094ba4566f84,28.561857,-80.577366,Cape Canaveral Space Force Station Space Launc...,20:00:00-04:00
5,2022-11-08,5e9e4501f509094ba4566f84,28.561857,-80.577366,Cape Canaveral Space Force Station Space Launc...,19:00:00-05:00
6,2022-11-14,5e9e4501f509094ba4566f84,28.561857,-80.577366,Cape Canaveral Space Force Station Space Launc...,19:00:00-05:00
7,2022-11-18,5e9e4502f509094188566f88,28.608058,-80.603956,Kennedy Space Center Historic Launch Complex 39A,17:00:00-05:00
8,2022-10-31,5e9e4501f509094ba4566f84,28.561857,-80.577366,Cape Canaveral Space Force Station Space Launc...,20:00:00-04:00
9,2022-10-31,5e9e4501f509094ba4566f84,28.561857,-80.577366,Cape Canaveral Space Force Station Space Launc...,20:00:00-04:00


## Get the user input on which launch to attend. (Consider if we can ask for cuisine preference here.)

In [24]:
# Ask the user to specify the index number of the launch they want.
user_input = input("Please select the index number of the launch you want to eat lunch around: ")

In [27]:
user_launch = df.iloc[int(user_input), :]
user_launch

date                                               2022-10-08
ID                                   5e9e4501f509094ba4566f84
lat                                                   28.5619
long                                                 -80.5774
location    Cape Canaveral Space Force Station Space Launc...
time                                           19:05:00-04:00
Name: 1, dtype: object